In [1]:
from collections import defaultdict

In [2]:
class Cave():
    @classmethod
    def from_gen(cls, gen, fix_ground_level: bool = False):
        cave = cls()
        for row in gen:
            row = row.rstrip()
            cave.add_rock_formation([
                tuple(map(int, pair.split(",")))
                for pair in row.split(" -> ")
            ])
        if fix_ground_level:
            cave.fix_ground_level()
        return cave

    def __getitem__(self, key: tuple[int, int]) -> int:
        x, y = key
        if self.ground_level is not None and y >= self.ground_level:
            return 1
        else:
            return self._state_dict[key]
    
    def __setitem__(self, key, val):
        self._state_dict[key] = val

    def __init__(self):
        self._state_dict = defaultdict(int)
        self[(500, 0)] = 2
        self._max_y = None
        self.ground_level = None
        self._char_rep = {
            -1: "~",
            0: ".",
            1: "#",
            2: "+",
            3: "o",
        }
    
    def __iter__(self):
        return iter(self._state_dict)

    def add_rock_formation(self, formation: list[tuple[int, int]]) -> None:
        prev_coords = None
        for x, y in formation:
            if prev_coords:
                prev_x, prev_y = prev_coords
                if x == prev_x:
                    step = 1 if prev_y < y else -1  
                    for tmp_y in range(prev_y, y+step, step):
                        self[(x, tmp_y)] = 1
                else:
                    step = 1 if prev_x < x else -1    
                    for tmp_x in range(prev_x, x+step, step):
                        self[(tmp_x, y)] = 1
            prev_coords = x, y
            self._max_y = max(self._max_y, y) if self._max_y else y
    
    def fix_ground_level(self, ground_level: int | None = None) -> None:
        if ground_level:
            self.ground_level = ground_level
        else:
            self.ground_level = self._max_y + 2

    def simulate_sand_unit(self) -> int:
        curr_x, curr_y = 500, 0

        max_val = max(self.ground_level, self._max_y) if self.ground_level else self._max_y
        while curr_y < max_val:
            if self[(curr_x, curr_y + 1)] <= 0:
                self[(curr_x, curr_y + 1)] = -1
                curr_x, curr_y = curr_x, curr_y + 1
            elif self[(curr_x - 1, curr_y + 1)] <= 0:
                self[(curr_x - 1, curr_y + 1)] = -1
                curr_x, curr_y = curr_x - 1, curr_y + 1
            elif self[(curr_x + 1, curr_y + 1)] <= 0:
                self[(curr_x + 1, curr_y + 1)] = -1
                curr_x, curr_y = curr_x + 1, curr_y + 1
            elif self[(curr_x, curr_y)] == -1:
                self[(curr_x, curr_y)] = 3
                return 0
            else:
                return 1
        return -1
    
    def __repr__(self) -> str:
        min_x = min(k[0] for k in self)
        min_y = min(k[1] for k in self)
        max_x = max(k[0] for k in self)
        max_y = max(k[1] for k in self)
    
        ret_str = ""
        for y in range(min_y, max_y + 1 + 3):
            ret_str += f"{y:3d} "
            for x in range(min_x, max_x + 1):
                ret_str += self._char_rep[self[(x, y)]]
            ret_str += "\n"
        return ret_str

In [3]:
def task_one(filename: str):
    with open(filename) as f:
        cave = Cave.from_gen(f)
    sum_var = 0
    while (res := cave.simulate_sand_unit()) == 0:
        sum_var += 1
    return sum_var, cave

In [4]:
task_one("test-input.txt")

(24,
   0 .......+...
   1 .......~...
   2 ......~o...
   3 .....~ooo..
   4 ....~#ooo##
   5 ...~o#ooo#.
   6 ..~###ooo#.
   7 ..~..oooo#.
   8 .~o.ooooo#.
   9 ~#########.
  10 ...........
  11 ...........
  12 ...........)

In [5]:
task_one("input.txt")

(1072,
   0 ........................................+.....................................
   1 ........................................~.....................................
   2 ........................................~.....................................
   3 ........................................~.....................................
   4 ........................................~.....................................
   5 ........................................~.....................................
   6 ........................................~.....................................
   7 ........................................~.....................................
   8 .......................................~o.....................................
   9 ......................................~ooo....................................
  10 .....................................~ooooo...................................
  11 ....................................~ooooooo....................

In [6]:
def task_two(filename: str):
    with open(filename) as f:
        cave = Cave.from_gen(f, fix_ground_level=True)
    sum_var = 0
    while (res := cave.simulate_sand_unit()) == 0:
        sum_var += 1
    if res == 1:
        sum_var += 1
    return sum_var, cave

In [7]:
task_two("test-input.txt")

(93,
   0 ..........+..........
   1 .........ooo.........
   2 ........ooooo........
   3 .......ooooooo.......
   4 ......oo#ooo##o......
   5 .....ooo#ooo#ooo.....
   6 ....oo###ooo#oooo....
   7 ...oooo.oooo#ooooo...
   8 ..oooooooooo#oooooo..
   9 .ooo#########ooooooo.
  10 ooooo.......ooooooooo
  11 #####################
  12 #####################
  13 #####################)

In [8]:
task_two("input.txt")

(24659,
   0 ..................................................................................................................................................................+..................................................................................................................................................................
   1 .................................................................................................................................................................ooo.................................................................................................................................................................
   2 ................................................................................................................................................................ooooo................................................................................................................................................................